In [1]:
from functions import *

greeks("NVDA", '2026-01-16', 175)

{'delta': -0.35240796141691444,
 'gamma': 0.017968381261557056,
 'vega': 0.2080392847061372,
 'theta': -0.10287411115973927,
 'rho': -0.06616729518844541}